## Step 2 Network Reconciliation
---
Use this notebook to setup a semi-automated reconciliation process between networks using functions available in 'conflation_tools.py' and 'network_reconcile.py.'

In general, you want to select one network to act as the base network (ground-truth) and add network data/attributes from the other networks.

These are the main functions in the conflation_tools module (type help(function_name) for a detailed description):
- match_nodes: finds node pairs between base and join network
- split_lines_create_points: Uses points from the join network to split links in the base network
- add_split_links: add the split links

From network_filter
- add_ref_ids: adds new reference ids from the nodes layer

Once finished reconciling, network can be exported for further manual reconciling or it can be prepped for network routing in BikewaySim.

type "help(insert_name_of_function)" to get more information about what the function does.

## Import Modules

In [1]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np

import networkx as nx
from tqdm import tqdm

import src.conflation_tools as conflation_tools
import src.add_attributes as add_attributes

## Adding and processing attribute data
These functions add in relevant attributes from the '.pkl' created in Step 1, or process supplemental data such as bicycle inventories.

The following three functions add attribute data back into the network and pre-processes it to match up with the desired impedance columns. These are custom network specific functions, so if adding a new network, will need to make another specific function.
- add_osm_attr
- add_here_attr
- add_abm_attr

# GDOT Project
---

### Add HERE road data to the OSM road data

In [2]:
# project directory
project_dir = Path.home() / 'Documents/BikewaySimData/Projects/gdot/networks'

In [3]:
osm_links = gpd.read_file(project_dir / 'filtered.gpkg',layer='osm_links')
osm_nodes = gpd.read_file(project_dir / 'filtered.gpkg',layer='osm_nodes')


In [5]:
osm_links = add_attributes.add_osm_attr(osm_links,project_dir / 'osm_attr.pkl')

In [6]:
# filter to roads
osm_road_links = osm_links[osm_links['link_type']=='road']
#osm_road_links = add_attributes.add_osm_attr(osm_road_links,project_dir / 'osm_attr.pkl')
osm_road_nodes = osm_nodes[osm_nodes['osm_N'].isin(osm_road_links['osm_A'].append(osm_road_links['osm_B']))]

In [7]:
#import here road layer
here_links = gpd.read_file(project_dir / 'filtered.gpkg',layer='here_links')
here_road_links = here_links[here_links['link_type']=='road']

#add attributes back
here_road_links = add_attributes.add_here_attr(here_road_links,project_dir / 'here_attr.pkl')

In [8]:
#function for adding attributes of one network network to another
road_links, overlapping = conflation_tools.add_attributes(
    osm_road_links, here_road_links, 'here', 100, 5, True)

Dissolving by 9 columns


This block modifies the street name attribute to compare the OSM street name vs the HERE assigned street name

In [9]:
street_names = dict(zip(osm_road_links['temp_ID'],osm_road_links['name']))
overlapping['name'] = overlapping['temp_ID'].map(street_names)

overlapping['match name'] = overlapping['ST_NAME'].apply(lambda row: conflation_tools.simplify_names(row))
overlapping['name'] = overlapping['name'].str.lower()


## Use these columns to examine the match quality

In [10]:
#check name
overlapping['name_check'] = overlapping['match name'] == overlapping['name']

#check overlap
overlapping['overlap_check'] = overlapping['percent_overlap'] > 0.9

#check bearing diff
overlapping['bearing_check'] = overlapping['bearing_diff'] < 5

#final check
overlapping['final_check'] = overlapping.apply(lambda row: row['name_check']+row['overlap_check']+row['bearing_check'],axis=1)

#drop 0s
overlapping = overlapping[overlapping['final_check'] >= 1]

#only keep max for each max
keep = overlapping.groupby('temp_ID')['final_check'].idxmax().to_list()
keep = overlapping.loc[keep]

## Add the here link id and export

In [11]:
#only keep here cols
remove_cols = set(osm_road_links.columns.tolist())
remove_cols.remove('temp_ID')
remove_cols = remove_cols & set(keep.columns.tolist())
keep.drop(columns=remove_cols,inplace=True)

In [12]:
# replace temp_id with the linkid
replace_temp_id = dict(zip(osm_road_links['temp_ID'],osm_road_links['osm_linkid']))
keep['osm_linkid'] = keep['temp_ID'].map(replace_temp_id)
keep.drop(columns=['temp_ID'],inplace=True)

In [13]:
osm_links = pd.merge(osm_links,keep,on='osm_linkid')

# osm_links.rename(columns={'osm_A':'A','osm_B':'B','osm_linkid':'linkid'},inplace=True)
# osm_nodes.rename(columns={'osm_N':'N'},inplace=True)

In [14]:
osm_links.to_file(project_dir/'reconciled.gpkg',layer='links')
#osm_nodes.to_file(project_dir/'reconciled.gpkg',layer='nodes')

In [15]:
# #import bike layer
# bike_links = merged[merged['link_type']=='bike']
# bike_nodes = gpd.read_file(project_dir / 'filtered.gpkg',layer='osm_nodes')
# bike_nodes = bike_nodes[bike_nodes['osm_N'].isin(bike_links['osm_A'].append(bike_links['osm_B']))]

In [16]:
# Hold off on this until the right before routing, this should only be for speeding up routing
#simplify the graph by removing interstital nodes
#merged = conflation_tools.remove_interstitial_nodes(osm_links,'osm_A','osm_B','osmid','osm_linkid',ignore_id=False)
# #Re-calculate the azimuth/bearing
# import pyproj
# prev_crs = merged.crs
# merged.to_crs('epsg:4326',inplace=True)
# merged[['fwd_azimuth','bck_azimuth']] = merged.apply(lambda row: modeling_turns.find_azimuth(row), axis=1)
# merged.to_crs(prev_crs,inplace=True)

# import network_filter

# nodes = gpd.read_file(project_dir / 'filtered.gpkg',layer='osm_nodes')

# #reassign link node ids
# ref_nodes_added = network_filter.add_ref_ids(merged,nodes,'osm')

In [17]:
# #get node count to find dead ends
# bike_nodes['num_links'] = bike_nodes['osm_N'].map(pd.concat([bike_links['osm_A'],bike_links['osm_B']],ignore_index=True).value_counts())
# dead_ends = bike_nodes[bike_nodes['num_links']==1]

# #remove dead ends already connected to road network
# dead_ends = dead_ends[-dead_ends['osm_N'].isin(osm_road_nodes['osm_N'])]

# #use full network to fix disconnected links from dead ends (road crossing breaks etc)
# osm_links = gpd.read_file(project_dir / 'filtered.gpkg',layer='osm_links')
# osm_nodes = gpd.read_file(project_dir / 'filtered.gpkg',layer='osm_nodes')
# connectors = conflation_tools.find_path(osm_links,osm_nodes,'osm',osm_road_nodes,dead_ends,100)

In [18]:
# connectors.explore()

In [19]:
# links = pd.concat([osm_road_links,connectors,bike_links],ignore_index=True).drop_duplicates()
# nodes = pd.concat([osm_road_nodes,bike_nodes],ignore_index=True)

In [20]:
# #create unique link id column (make sure to find the max linkid using the full dataset)
# max_linkid = int(links['osm_linkid'].max())
# links.loc[links['osm_linkid'].isna(),'osm_linkid'] = range(max_linkid+1,max_linkid+links['osm_linkid'].isna().sum()+1)

## Remove isolated nodes/links

In [21]:
# before_links = links.shape[0]
# before_nodes = nodes.shape[0]

# #create undirected graph
# G = nx.Graph()  # create directed graph
# for row in links[['osm_A','osm_B']].itertuples(index=False):
#     # forward graph, time stored as minutes
#     G.add_edges_from([(row[0],row[1])])

# #only keep largest component
# largest_cc = max(nx.connected_components(G), key=len)

# #get nodes
# nodes = nodes[nodes['osm_N'].isin(largest_cc)]
# #get links
# links = links[links['osm_A'].isin(largest_cc) & links['osm_B'].isin(largest_cc)]

# print('Links removed:',before_links-links.shape[0],'Nodes removed:',before_nodes-nodes.shape[0])

In [22]:
# links.rename(columns={'osm_A':'A','osm_B':'B','osm_linkid':'linkid'},inplace=True)
# nodes.rename(columns={'osm_N':'N'},inplace=True)

In [23]:
# #export
# links.to_file(project_dir / 'reconciled_network.gpkg',layer='links')
# nodes.to_file(project_dir / 'reconciled_network.gpkg',layer='nodes')

# Assessing Bike-Transit Accessibility
---
The code blocks below this are for creating a network to use for transitsim. Only uses OSM.

In [24]:
# # project directory
# project_dir = Path.home() / 'Documents/TransitSimData/Data/networks'

# #import osm road layer
# osm_road_links = gpd.read_file(project_dir / 'filtered.gpkg',layer='osm_links')
# osm_road_links = osm_road_links[osm_road_links['link_type']=='road']
# osm_road_nodes = gpd.read_file(project_dir / 'filtered.gpkg',layer='osm_nodes')
# osm_road_nodes = osm_road_nodes[osm_road_nodes['osm_N'].isin(osm_road_links['osm_A'].append(osm_road_links['osm_B']))]

# #import bike layer
# bike_links = gpd.read_file(project_dir / 'filtered.gpkg',layer='osm_links')
# bike_links = bike_links[bike_links['link_type']=='bike']
# bike_nodes = gpd.read_file(project_dir / 'filtered.gpkg',layer='osm_nodes')
# bike_nodes = bike_nodes[bike_nodes['osm_N'].isin(bike_links['osm_A'].append(bike_links['osm_B']))]

# #get node count to find dead ends
# bike_nodes['num_links'] = bike_nodes['osm_N'].map(pd.concat([bike_links['osm_A'],bike_links['osm_B']],ignore_index=True).value_counts())
# dead_ends = bike_nodes[bike_nodes['num_links']==1]

# #remove dead ends already connected to road network
# dead_ends = dead_ends[-dead_ends['osm_N'].isin(osm_road_nodes['osm_N'])]

# #use full network to fix disconnected links from dead ends (road crossing breaks etc)
# #connectors are assigned a new unique link id before export
# osm_links = gpd.read_file(project_dir / 'filtered.gpkg',layer='osm_links')
# osm_nodes = gpd.read_file(project_dir / 'filtered.gpkg',layer='osm_nodes')
# connectors = conflation_tools.find_path(osm_links,osm_nodes,'osm',osm_road_nodes,dead_ends,50)

# #add connectors, bike links, and the new nodes
# links = pd.concat([osm_road_links,connectors,bike_links],ignore_index=True).drop_duplicates()
# nodes = pd.concat([osm_road_nodes,bike_nodes],ignore_index=True).drop_duplicates()
# #create unique link id column (make sure to find the max linkid using the full dataset)
# max_linkid = int(osm_links['osm_linkid'].max())
# links.loc[links['osm_linkid'].isna(),'osm_linkid'] = range(max_linkid+1,max_linkid+links['osm_linkid'].isna().sum()+1)
# #add attributes
# links = add_osm_attr(links, project_dir / 'osm_attr.pkl')

# #export
# links.to_file(project_dir / 'reconciled_network.gpkg',layer='links')
# nodes.to_file(project_dir / 'reconciled_network.gpkg',layer='nodes')